<a href="https://colab.research.google.com/github/Ranadip213/Detecting_Multimodal_Hate_Speech_in_Internet_Memes/blob/main/memes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import pandas as pd
import re
import nltk
import zipfile
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 1: Extract the photos from the zip file
with zipfile.ZipFile('/content/drive/MyDrive/dataset/archive.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/photos')  # Extract to a folder named 'photos'

In [ ]:
# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Define the path to your image dataset folder
photos_folder = '/content/photos/harmful_meme_politics/images'

In [ ]:
# Preprocess image data (e.g., resize, normalize pixel values)
def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, target_size)  # Resize image to target size
    image = image / 255.0  # Normalize pixel values to range [0, 1]
    return image

In [ ]:
# Preprocess all images in the dataset folder
preprocessed_images = []
for filename in os.listdir(photos_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(photos_folder, filename)
        preprocessed_image = preprocess_image(image_path)
        if preprocessed_image is not None:
            preprocessed_images.append(preprocessed_image)

In [ ]:

# Load the CSV file containing text data
csv_file_path = '/content/photos/harmful_meme_politics/harmful_meme_for_politics.csv'
text_data = pd.read_csv(csv_file_path)

In [ ]:
def preprocess_text(text):
    # Check if the text is not NaN
    if pd.notna(text):
        # Convert text to lowercase
        text = text.lower()

        # Remove non-alphanumeric characters and extra whitespaces
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        text = re.sub(r'\s+', ' ', text)

        # Tokenize text into words
        words = word_tokenize(text)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if word not in stop_words]

        # Lemmatize words
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words]

        # Combine words back into a single string
        preprocessed_text = ' '.join(words)

        return preprocessed_text
    else:
        return ''  # Return an empty string for missing values


# Preprocess text data from the CSV file
text_data['labels'] = text_data['labels'].apply(preprocess_text)

In [ ]:
# Create a DataFrame for the combined data
combined_data = pd.concat([text_data, pd.DataFrame({'image': preprocessed_images})], axis=1)

In [ ]:

# Preprocess text data from the CSV file
text_data['text'] = text_data['text'].apply(preprocess_text)

In [ ]:
# Step 3: Combine preprocessed text and image data

# Ensure the lengths match before combining
if len(text_data) != len(preprocessed_images):
    print("Error: Lengths of text and image data do not match.")
else:
    # Create a DataFrame for the combined data
    combined_data = pd.concat([text_data, pd.DataFrame({'image': preprocessed_images})], axis=1)


In [ ]:
# Step 4: Split the dataset into training and test sets (80% training, 20% testing)

X_train, X_test, y_train, y_test = train_test_split(combined_data[['text', 'image']], combined_data['labels'], test_size=0.2, random_state=42)


In [ ]:
# Step 5: Define and train a multimodal machine learning model

# Define a model architecture (e.g., support vector machine)
model = make_pipeline(StandardScaler(), SVC(kernel='linear', probability=True))

# Train the model on the training data
model.fit(X_train, y_train)

ValueError: could not convert string to float: 'generating enough angry white guy stay business long term lindsey graham fturs af apubican party scandal free republican running raised druggie kidsno ominat indictment nota tul mouhed ashole barely regstering pole republican volers prefer criminal scunbag lindsey graham meme trump meme library flickr'